In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos

import numpy as np
import time
import datetime

from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

In [ ]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TERMINATE_WALLET]

In [ ]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False)
vega.start()

#### Create the wallets

In [ ]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

#### Create the asset

In [ ]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

In [ ]:
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.providers.fee.distributionTimeStep", "600s")
vega.wait_for_datanode_sync()
vega.update_network_parameter(MM_WALLET.name, "market.fee.factors.infrastructureFee", "0.0")

In [ ]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [ ]:
vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e8
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

In [ ]:
market_decimals = 3 
position_decimals = 2

#### Create the market and set up liquidity

In [ ]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=position_decimals,
        market_decimals=market_decimals,
    )

In [ ]:
market_id = vega.all_markets()[0].id
print("Market id = " + market_id)

In [ ]:
cumulative_sum_after_mint = 0.0 
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    cumulative_sum_after_mint +=  general + margin + bond 

print("cumulative sum = " + str(cumulative_sum_after_mint))


In [ ]:
MM_commintment_amount=50000
vega.submit_liquidity(
        MM_WALLET.name,
        market_id=market_id,
        commitment_amount=MM_commintment_amount,
        fee=0.05,
        buy_specs=[
            ("PEGGED_REFERENCE_BEST_BID", 0.5, 1), 
            ("PEGGED_REFERENCE_BEST_BID", 1.0, 1)],
        sell_specs=[
            ("PEGGED_REFERENCE_BEST_ASK", 0.5, 1),
            ("PEGGED_REFERENCE_BEST_ASK", 1.0, 1),
        ],
        is_amendment=True,
    )
vega.wait_fn(1)
vega.wait_for_total_catchup()



In [ ]:
# Submit orders which will stay on the book and create best bid / ask 
vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=96,
    wait=False,
)

vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=104,
    wait=False,
)



In [ ]:
num_levels = 11
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print("ask prices: "+str(ask_prices))
print("ask volumes: "+str(ask_volumes))
print("--------------------------------------------------------")
print("bid prices: "+str(bid_prices))
print("bid volumes: "+str(bid_volumes))


#### Do some trading

In [ ]:
# Do a trade which will cross
price = 99
liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market undistributed liquidity fees: " + str(liq_fees))
vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price,
    wait=True,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price,
    wait=True,
)
liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market undistributed liquidity fees: " + str(liq_fees))
vega.wait_fn(1)
vega.wait_for_total_catchup()
liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market undistributed liquidity fees: " + str(liq_fees))

In [ ]:
num_levels = 11
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print("ask prices: "+str(ask_prices))
print("ask volumes: "+str(ask_volumes))
print("--------------------------------------------------------")
print("bid prices: "+str(bid_prices))
print("bid volumes: "+str(bid_volumes))


In [ ]:

liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market pretrade undistributed liquidity fees: " + str(liq_fees))

vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price,
    wait=True,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price,
    wait=True,
)
liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market post trade undistributed liquidity fees: " + str(liq_fees))
vega.forward('1s')

market_data = vega.market_data(market_id=market_id)
print("Market OI is: {}".format(market_data.open_interest))

In [ ]:
num_levels = 11
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print("ask prices: "+str(ask_prices))
print("ask volumes: "+str(ask_volumes))
print("--------------------------------------------------------")
print("bid prices: "+str(bid_prices))
print("bid volumes: "+str(bid_volumes))

liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market undistributed liquidity fees: " + str(liq_fees))

In [ ]:

vega.wait_for_total_catchup()

market_data = vega.market_data(market_id=market_id)

best_static_bid = float(market_data.best_static_bid_price)/10**market_decimals
best_static_ask = float(market_data.best_static_offer_price)/10**market_decimals
print("best static bid price: "+ str(best_static_bid)+", volume: "+str(float(market_data.best_static_bid_volume)/10**position_decimals))
print("best static ask price: "+ str(best_static_bid)+", volume: "+str(float(market_data.best_static_offer_volume)/10**position_decimals))
markprice = float(market_data.mark_price) / 10**market_decimals
print("Mark price is "+str(markprice))

liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market undistributed liquidity fees: " + str(liq_fees))
print("Market OI is: {}".format(market_data.open_interest))

print("And positions for trader 1 are: {}".format(vega.positions_by_market(market_id=market_id, wallet_name=TRADER_1_WALLET.name)))
print("And positions for trader 2 are: {}".format(vega.positions_by_market(market_id=market_id, wallet_name=TRADER_2_WALLET.name)))
print("And positions for MM are: {}".format(vega.positions_by_market(market_id=market_id, wallet_name=MM_WALLET.name)))

In [ ]:
vega.wait_fn(1)
vega.wait_for_total_catchup()

liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)

cumulative_sum = 0.0 
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    cumulative_sum +=  general + margin + bond 
    
print("Pre-Settlement Market undistributed liquidity fees: " + str(liq_fees))


In [ ]:
vega.settle_market(settlement_wallet = TERMINATE_WALLET.name, settlement_price = 95, market_id=market_id)

In [ ]:
vega.wait_fn(1)
vega.wait_for_total_catchup()

liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)

cumulative_sum = 0.0 
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    cumulative_sum +=  general + margin + bond 
    
print("Market undistributed liquidity fees: " + str(liq_fees))
print("cumulative sum = " + str(cumulative_sum))
print("initially minted = " + str(cumulative_sum_after_mint))
print("cumulative sum + undistributed liq fees = " + str(cumulative_sum + liq_fees)) 

if (np.abs(liq_fees - 4.95) > 1e-2):
    raise Exception("Expected to see 5.02503 - this isn't correct but it's the bug I am expecting to demonstrate")

In [ ]:
print("Check out http://localhost:"+str(vega.data_node_graphql_port))
print("Market id = " + market_id)

#vega.stop()

In [ ]:
vega.data_node_postgres_port

In [ ]:

market_data_post_settle = vega.market_info(market_id=market_id)

In [ ]:
# Select your transport with a defined url endpoint
graphql_server_addr="http://localhost:"+str(vega.data_node_graphql_port) + "/query"
transport = AIOHTTPTransport(url=graphql_server_addr)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

query = gql(
    """
    {
        markets {
            id
        }
    }
    """
)

# Execute the query on the transport
result = await client.execute_async(query) 
print(result)